##Setting

In [1]:
!git clone https://github.com/openaifab/suicide_multiclass.git
#!pip install ckiptagger

Cloning into 'suicide_multiclass'...
remote: Enumerating objects: 22, done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 22
Unpacking objects: 100% (22/22), done.


In [2]:
#%tensorflow_version 1.X
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
%cd suicide_multiclass
%pwd

/content/suicide_multiclass


'/content/suicide_multiclass'

#Data preprocess

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle

In [5]:
from OpenFabLibrary import JeibaCutWords
from OpenFabLibrary import process_df

##Set parameters

In [6]:
data_dir = "/".join((".", "data"))
#用記事本打開csv文件，另存為设置編碼為utf-8
data_source = "danger_words.csv"

##Load danger word data

In [7]:
data_df = pd.read_csv(data_dir + '/' + data_source, encoding='utf8', delimiter=',')
print(data_df.head())

   ID           Description  Class
0   0             不太想活 自殺方法      2
1   1           不留戀這世界 厭世規劃      2
2   2               傷害自己 絕望      2
3   3  命是網友救起來 手腕 傷痕 鬼門關 救命      2
4   4           思覺失調症 幻聽 餓死      2


##斷詞斷句

In [8]:
# 斷詞處理
cut_data_df = JeibaCutWords(data_df, "Description")

# word count
sentence_data = cut_data_df.loc[:,'sentence']
cut_data_df['word_count'] = sentence_data.str.len() 

print(cut_data_df.head())

Building prefix dict from /content/suicide_multiclass/Jeiba/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ucb66ba17c4739edbeb5fb122bb3a9bb9.cache
Loading model cost 1.594 seconds.
Prefix dict has been built successfully.


   id                          sentence  class  word_count
0   0                     [不太想, 自殺, 方法]      2           3
1   1              [不留, 留戀, 世界, 厭世, 規劃]      2           5
2   2                          [傷害, 絕望]      2           2
3   3  [命, 網友, 救起, 手腕, 傷痕, 鬼門, 鬼門關, 救命]      2           8
4   4           [思, 覺, 失調, 失調症, 幻聽, 餓死]      2           6


##Generate danger word list

In [9]:
corpus = process_df(cut_data_df)
print(f"total cases: {len(corpus)}")
print(corpus)

total cases: 112
[['不太想 自殺 方法', 2], ['不留 留戀 世界 厭世 規劃', 2], ['傷害 絕望', 2], ['命 網友 救起 手腕 傷痕 鬼門 鬼門關 救命', 2], ['思 覺 失調 失調症 幻聽 餓死', 2], ['想死 棺材', 2], ['放棄 人生 陰陽 兩極 想 離開 死路', 2], ['放棄 拯救', 2], ['死去 屍體 墳墓 死 不遠', 2], ['死法', 2], ['需要 存在 在於 世上', 2], ['活到 一天 消失', 2], ['苟活 真的 好 想死', 2], ['x 傾向', 2], ['自我 毀滅', 2], ['自殺 求救 。,', 2], ['不好 想法 真的 做', 2], ['想死', 1], ['死亡 尋死', 1], ['下輩 下輩子 輩子', 1], ['不知 知道 再 走 遠', 1], ['宣告 剩下 日子 裡 倒數 數計 計時', 1], ['走 裡 地方 真的 好 想 離開', 1], ['希望 能夠 結束', 1], ['掐住 脖子 窒息 亡', 1], ['死 後 瀕死', 1], ['腐爛 空殼 魂魄 死活', 1], ['結束 上天 劈 死', 1], ['自殺', 1], ['解脫', 1], ['活下去 下去', 0], ['厭世', 0], ['索然 索然乏味 乏味', 0], ['不太想 自殺 方法', 2], ['不留 留戀 累', 2], ['藥物 傷害 掉下去 下去 恨 絕望', 2], ['傷心 健忘 手腕 傷痕 鬼門 鬼門關 關門 門口 拉回 拉回來 回來 找 關愛 同性 陰天 、,', 2], ['藥 餓死 算了', 2], ['想死', 2], ['痛苦 陰陽 兩極', 2], ['垃圾 無 藥 可救', 2], ['有期 期待 有傷 傷害 死 屍體', 2], ['死法', 2], ['需要 存在 在於 世上 價值', 2], ['桎梏 憂鬱 社 恐 沒錢 恐懼 活到 一天 消失', 2], ['苟活 死 安眠 安眠藥 眠藥 悠 丁 哭 憂鬱 失 好爛 好 糟糕 爛', 2], ['x 傾向', 2], ['想 自我 毀滅', 2], ['地獄 安 PTSD 眠藥 自殺 失控', 2], ['藥 不好 想法 失望', 2], 

In [10]:
danger_words = []
#for i in range(len(corpus)):
for i in range(10):
  temp = corpus[i][0].split(' ')
  for j in range(len(temp)):
    if temp[j] not in danger_words:
      danger_words.append(temp[j])

print(danger_words)

['不太想', '自殺', '方法', '不留', '留戀', '世界', '厭世', '規劃', '傷害', '絕望', '命', '網友', '救起', '手腕', '傷痕', '鬼門', '鬼門關', '救命', '思', '覺', '失調', '失調症', '幻聽', '餓死', '想死', '棺材', '放棄', '人生', '陰陽', '兩極', '想', '離開', '死路', '拯救', '死去', '屍體', '墳墓', '死', '不遠', '死法']


##危險因子儲存

In [11]:
# 儲存為pickle格式檔案
with open(data_dir + '/' + 'danger_words', 'wb') as file:
    pickle.dump(danger_words, file)

In [12]:
# load article tokenized
with open(data_dir + "/" + 'danger_words', 'rb') as file:
    temp = pickle.load(file)

In [13]:
print(temp)

['不太想', '自殺', '方法', '不留', '留戀', '世界', '厭世', '規劃', '傷害', '絕望', '命', '網友', '救起', '手腕', '傷痕', '鬼門', '鬼門關', '救命', '思', '覺', '失調', '失調症', '幻聽', '餓死', '想死', '棺材', '放棄', '人生', '陰陽', '兩極', '想', '離開', '死路', '拯救', '死去', '屍體', '墳墓', '死', '不遠', '死法']
